How well do our observations at 0.33$R_{26}$ / 0.4$R_{26}$ match MaNGA's observations at the same radius?  And how well do our velocities observed at these radii reproduce the asymptotic rotational velocity?

In [20]:
import numpy as np

from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.constants as const
import astropy.units as u

In [36]:
h = 1
H0 = 100*h

c = const.c.to('km/s')

q0 = 0.2

MANGA_SPAXEL_SIZE = 0.5*u.arcsec

# Import data

## DESI

In [9]:
tf_targets = Table.read('/global/cfs/projectdirs/desi/science/td/pv/desi_pv_tf_fuji_healpix.fits')

## SGA

In [10]:
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')

In [16]:
SGA_dict = {}

for i in range(len(SGA)):
    
    SGA_dict[SGA['SGA_ID'][i]] = i

## SDSS MaNGA cross-match file

Made by Grace Chiodo in Summer 2021, using SDSS MaNGA DR15.

In [23]:
SGA_MaNGA = Table.read('MaNGA_DESI_galaxies_08042021.txt', format='ascii.commented_header')

SGA_MaNGA[:5]

MaNGA_plate,MaNGA_IFU,sga_id,BA,PA,plateifu
int64,int64,int64,float64,float64,str10
7968,3702,97758,0.3215293288230896,159.22218322753906,7968-3702
8479,6101,433955,0.32715362310409546,54.33457565307617,8479-6101
8479,6102,487085,0.5813088417053223,1.3182913064956665,8479-6102
8480,6103,487085,0.5813088417053223,1.3182913064956665,8480-6103
8481,12705,810971,0.268082857131958,56.5462760925293,8481-12705


## SDSS MaNGA best-fits

From Douglass & Demina (2022)

In [12]:
#MaNGA_fits = Table.read('DESI_vel_map_results_BB_smooth_lt_1p85_v2.txt', 
MaNGA_fits = Table.read('DRP-master_file_vflag_BB_smooth1p85_mapFit_N2O2_HIdr2_morph_SK_H2_v6.txt', 
                        format='ascii.commented_header')

# Compare DESI's observed rotational velocities at 0.33$R_{26}$ with that calculated from the best-fit to the MaNGA data at that radius

## 1) Calculate the rotational velocities for the DESI galaxies

### Separate the fuji data into center and off-center observations

In [13]:
def firstdigit(n):
    """
    Return the first digit of a number.
    
    Parameters
    ----------
    n : int, float, or ndarray
        Number or list of numbers.
    
    Returns
    -------
    digit : int
        First digit of the number.
    """
    
    digit = np.trunc(n * 10**(-np.trunc(np.log10(n)))).astype(int)
    
    return digit

In [14]:
first_digits = firstdigit(tf_targets['TARGETID'])

centers = tf_targets[first_digits == 3]
axis = tf_targets[first_digits != 3]

/tmp/ipykernel_83662/917620242.py:16: RuntimeWarning: invalid value encountered in log10
  digit = np.trunc(n * 10**(-np.trunc(np.log10(n)))).astype(int)


### Clean the fuji center observations

Only keep those with
 * DELTACHI2 > 25
 * ZWARN == 0

In [15]:
good_centers = centers[(centers['DELTACHI2'] > 25) & (centers['ZWARN'] == 0)]

### If an SGA galaxy has at least one observation at its center, set the redshift of that galaxy

In [17]:
SGA['Z_DESI'] = np.nan
SGA['ZERR_DESI'] = np.nan

for sga_id in np.unique(good_centers['SGA_ID']):
    
    # Find the row in SGA for this galaxy
    SGA_idx = SGA_dict[sga_id]
    
    # Set the redshift of this galaxy to be the average redshift of all good center observations
    SGA['Z_DESI'][SGA_idx] = np.mean(good_centers['Z'][good_centers['SGA_ID'] == sga_id])
    SGA['ZERR_DESI'][SGA_idx] = np.mean(good_centers['ZERR'][good_centers['SGA_ID'] == sga_id]**2)/np.sum(good_centers['SGA_ID'] == sga_id)

### Calculate the rotational velocity

In [37]:
axis['SKY_FIBER_DIST'] = 0.
axis['SKY_FIBER_DIST_R26'] = 0.
axis['V_ROT'] = np.nan
axis['V_ROT_ERR'] = np.nan


# For each SGA galaxy that has at least one center observation, calculate the 
# distance for all of that galaxy's targets
for sga_gal in np.unique(centers['SGA_ID']):
    
    # Identify all galaxy targets on this galaxy
    obs_idx = axis['SGA_ID'] == sga_gal
    
    # Find galaxy index in SGA catalog
    sga_idx = SGA_dict[sga_gal]
    
    #---------------------------------------------------------------------------
    # Calculate distance between each observation and the center
    #---------------------------------------------------------------------------
    center_coords = SkyCoord(ra=SGA['RA'][sga_idx], 
                             dec=SGA['DEC'][sga_idx], 
                             unit=u.degree)
    target_coords = SkyCoord(ra=axis['RA'][obs_idx], 
                             dec=axis['DEC'][obs_idx], 
                             unit=u.degree)
    
    sep2d = target_coords.separation(center_coords)
    
    axis['SKY_FIBER_DIST'][obs_idx] = sep2d
    axis['SKY_FIBER_DIST_R26'][obs_idx] = 2*sep2d.to('arcmin')/(SGA['D26'][sga_idx]*u.arcmin)
    #---------------------------------------------------------------------------
    
    
    #---------------------------------------------------------------------------
    # Calculate rotational velocity
    #---------------------------------------------------------------------------
    # Use the average redshift of all center observations for the systemic velocity
    z_center = np.mean(SGA['Z_DESI'][sga_idx])
    z_err_center2 = SGA['ZERR_DESI'][sga_idx]**2

    # Calculate rotational velocity for all observations of the galaxy
    axis['V_ROT'][obs_idx] = c*(axis['Z'][obs_idx] - z_center)
    axis['V_ROT_ERR'][obs_idx] = c*np.sqrt(axis['ZERR'][obs_idx]**2 + z_err_center2)
    #---------------------------------------------------------------------------
    
    
    #---------------------------------------------------------------------------
    # Correct rotational velocities for inclination angle
    #---------------------------------------------------------------------------
    cosi2 = (SGA['BA'][sga_idx]**2 - q0**2)/(1 - q0**2)
    
    # Galaxies with b/a < q0
    if cosi2 < 0:
        cosi2 = 0
    
    axis['V_ROT'][obs_idx] /= np.sin(np.arccos(np.sqrt(cosi2)))
    #---------------------------------------------------------------------------

/tmp/ipykernel_83662/116782489.py:56: RuntimeWarning: divide by zero encountered in true_divide
  axis['V_ROT'][obs_idx] /= np.sin(np.arccos(np.sqrt(cosi2)))


### Just keep those velocities measured at 0.33$R_{26}$

In [38]:
axis_0p3 = axis[(axis['SKY_FIBER_DIST_R26'] > 0.3) & (axis['SKY_FIBER_DIST_R26'] < 0.4)]

### Remove "bad" galaxies

Those with
* $V > 1000$ km/s

In [39]:
axis_0p3_good = axis_0p3[np.abs(axis_0p3['V_ROT']) < 1000]

axis_0p3_good[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,V_ROT,V_ROT_ERR
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64,float64,float64,float64,float64
1070235180335113,208.919635503634,4.98868419056778,26090,sv3,0.00406957966391755,8.79850271950027e-05,4,0.232979318086109,fuji/healpix/sv3/bright/260/26090/redrock-sv3-bright-26090.fits,EXT,631395,208.91963550363434,4.988684190567775,0.01104459258096158,0.3300000131125383,57.33193100056125,26.377247569986736
1083429319868425,208.919635503634,4.98868419056778,26090,sv3,0.00414750849232355,7.13762071499604e-05,0,37.3058008448807,fuji/healpix/sv3/dark/260/26090/redrock-sv3-dark-26090.fits,EXT,631395,208.91963550363434,4.988684190567775,0.01104459258096158,0.3300000131125383,82.45297543020575,21.398048584203842
1083338303471619,178.909990284283,1.23197064456541,27650,sv3,0.00651958780483995,6.84625673525266e-06,0,478.315322757233,fuji/healpix/sv3/dark/276/27650/redrock-sv3-dark-27650.fits,EXT,1033344,178.90999028428269,1.2319706445654142,0.008001821639844116,0.3300000001109575,121.97667415638571,2.052456134762253
1083338303471618,178.897801871515,1.24234505754909,27650,sv3,0.00612525938757635,5.59810860736493e-06,0,800.964292295277,fuji/healpix/sv3/dark/276/27650/redrock-sv3-dark-27650.fits,EXT,1033344,178.89780187151533,1.242345057549085,0.008001821639837198,0.33000000011067215,-77.4715539900934,1.6782707395550946
1083435338694660,208.737562035713,5.31217069943004,26432,sv3,0.00435515438171736,5.95161028988655e-05,0,64.5522083645719,fuji/healpix/sv3/dark/264/26432/redrock-sv3-dark-26432.fits,EXT,547778,208.7375620357128,5.31217069943004,0.02245240582779445,0.33000001952950575,-82.94586723919,17.842478778633268


### Add these velocities to the SDSS MaNGA - SGA cross-match file

In [40]:
SGA_MaNGA['V_0p33R26'] = np.nan
SGA_MaNGA['V_0p33R26_ERR'] = np.nan

weights = 1./(axis_0p3_good['V_ROT_ERR']**2)

for i in range(len(SGA_MaNGA)):
    
    # Does this galaxy have any observations?
    i_obs = axis_0p3_good['SGA_ID'] == SGA_MaNGA['sga_id'][i]
    
    if np.sum(i_obs) > 0:
        
        # Average all velocities at this radius
        SGA_MaNGA['V_0p33R26'][i] = np.average(np.abs(axis_0p3_good['V_ROT'][i_obs]), 
                                               weights=weights[i_obs])
        SGA_MaNGA['V_0p33R26_ERR'][i] = np.sqrt(1./np.sum(weights[i_obs]))

In [42]:
SGA_MaNGA[np.isfinite(SGA_MaNGA['V_0p33R26'])].show_in_notebook()

idx,MaNGA_plate,MaNGA_IFU,sga_id,BA,PA,plateifu,V_0p33R26,V_0p33R26_ERR
0,8479,6102,487085,0.5813088417053223,1.3182913064956665,8479-6102,223.95494076075573,8.795513663749514
1,8480,6103,487085,0.5813088417053223,1.3182913064956665,8480-6103,223.95494076075573,8.795513663749514
2,8481,12705,810971,0.268082857131958,56.5462760925293,8481-12705,143.9510078418774,2.215518265835735
3,8484,3701,302062,0.6225041747093201,119.090087890625,8484-3701,109.17612316124635,2.1124446051803405
4,8547,6102,925354,0.6025776267051697,146.05860900878906,8547-6102,205.77258114621176,1.5235238718905617
5,8547,6103,171058,0.5511276125907898,3.9989326000213623,8547-6103,138.28308158423158,0.4916285418325515
6,8555,12703,127268,0.2092917561531067,143.2239532470703,8555-12703,210.96779881214266,2.332334252430736
7,8555,1902,575284,0.6609105467796326,106.39098358154297,8555-1902,137.07386885858102,4.085461504059483
8,8555,9102,1285122,0.7685890793800354,107.45079040527344,8555-9102,162.57738944564244,1.358729053158345
9,8600,1901,186915,0.7746372222900391,129.28488159179688,8600-1901,166.8309402755933,1.5433379347652003
